# Fourier Winding Widget

> Por Santiago Henao Castellanos para el curso Métodos Computacionales 2

In [25]:
import ipywidgets as widgets
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import ellipj, ellipk

@njit("f8[:](f8[:],f8[:],f8,f8,f8)",cache=True,fastmath=True)
def NUDFT_grid(t_arg,m_arg,f_ini,f_end,df):
    """Non-Uniform Discrete Fourier Transform on an evenly-spaced frequency grid
    with the Kurtz (1985, MNRAS 213,773) algorithm"""
    dF = np.exp(-2j*np.pi*t_arg*df)
    F = np.zeros_like(np.arange(f_ini,f_end,df))
    curl = m_arg * np.exp(-2j*np.pi*t_arg*f_ini)
    F[0] = np.abs(np.sum(curl))**2
    for i in range(1,len(F)):
        curl = curl*dF
        F[i] = np.abs(np.sum(curl))**2
    return F

In [28]:
def Fourier_winding_widget(winding_line=True,freq=0.0,t_max=1.0,t_step=0.01,function_shape=0.98,y_0=0.,f_max=1.,f_step=0.001):
    axd = plt.figure(layout="constrained",figsize=(12,4)).subplot_mosaic("ABB\nACC",width_ratios=[2,1,4])
    ts = np.arange(0,t_max,t_step)
    ys, *_  = ellipj(ts,function_shape)
    ys += y_0
    f_spec = (0,f_max,f_step)
    fs = np.arange(*f_spec)
    Fs = NUDFT_grid(ts,ys,*f_spec)
    Z = ys * np.exp(-2j*np.pi*ts*freq)

    axd["A"].plot(Z.real,Z.imag,"."+("-" if winding_line else " ")+"k",alpha=0.4)
    axd["A"].scatter(Z.mean().real,Z.mean().imag,c='r',s=200,zorder=10,alpha=0.4)
    axd["A"].grid(True)
    axd["A"].set_xlim(-2.5,2.5)
    axd["A"].set_ylim(-2.5,2.5)
    axd["A"].set_xlabel("Re(Z)")
    axd["A"].set_ylabel("Im(Z)")
    axd["A"].set_title(r'$ Z(f,t) = y(t) \exp\left(-2\pi i t f\right)$')
    axd["B"].plot(ts,ys,".-k")
    axd["B"].margins(x=0)
    axd["B"].set_xlabel("Time $t$")
    axd["B"].set_ylabel("Intenstiy $y(t)$")
    for t_cut in np.arange(0,t_max,1/freq):
        axd["B"].axvline(t_cut,c='r',ls="--",alpha=0.6)
    axd["C"].plot(fs,np.sqrt(Fs)/len(ts),c='k')
    #axd["C"].set_yscale("log")
    axd["C"].margins(x=0)
    axd["C"].axvline(freq,c='r',label="Current winding frequency")
    axd["C"].set_xlabel("Frequency $f$")
    axd["C"].set_ylabel(r" $\left| \frac{1}{N} \sum_{k=0}^{N-1} Z(f,t_k) \right|$")
    axd["C"].legend()

    plt.show()


In [30]:
width="1200px"
layout = widgets.Layout(width=width)
widgets.interact(Fourier_winding_widget,
    winding_line=True,
    freq=widgets.FloatSlider(           value=0.075,    min=0,      max=0.4,    step=0.001,layout=layout),
    t_max=widgets.FloatSlider(          value=60,       min=1,      max=100,    step=0.5,layout=layout),
    t_step=widgets.FloatSlider(         value=0.1,      min=0.001,  max=2,      step=0.001,layout=layout),
    function_shape=widgets.FloatSlider( value=0.98,     min=0.001,  max=0.999,  step=0.001,layout=layout),
    y_0=widgets.FloatSlider(            value=1.2,      min=0,      max=2,      step=0.1,layout=layout),
    f_max=widgets.FloatSlider(          value=0.4,      min=0.1,    max=10,     step=0.1,layout=layout),
    f_step=widgets.FloatSlider(         value=0.001,    min=0.0002, max=0.1,    step=0.002,layout=layout),
);

interactive(children=(Checkbox(value=True, description='winding_line'), FloatSlider(value=0.075, description='…